<a href="https://colab.research.google.com/github/qcuongning/collab/blob/main/Thyroid_nodule_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Với các hàm cần sử dụng backend Keras mà bị lỗi K not contribute cần đổi về keras==2.2.4


The codebase is heavily inspired by the [respotory](https://github.com/qubvel/efficientnet)



# Get Data

In [24]:
import os
import numpy as np
from matplotlib import pyplot as plt
import json
import cv2
import csv
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import applications as app
seed = 1234
np.random.seed(seed)
from zipfile import ZipFile


In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

KeyboardInterrupt: ignored

In [ ]:
file_id = '1bbKAqUuk7Y1q3xsDSwP07oOXN_GL3SQM'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('COVID-SemiSegz')
with ZipFile('COVID-SemiSegz', 'r') as tt:
  tt.extractall("")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#su dung khi dataset o trong drive neu khong bo qua
%cp drive/My\ Drive/COVID-SemiSeg.zip /content/
with ZipFile('COVID-SemiSeg.zip', 'r') as tt:
  tt.extractall("")

In [ ]:
#su dung khi dataset o trong drive neu khong bo qua
%cp drive/My\ Drive/thyroid/training_data.zip /content/
with ZipFile('training_data.zip', 'r') as tt:
  tt.extractall("")

In [ ]:
file_name = 'weights_save'
if not os.path.exists(file_name):
  os.mkdir(file_name)

# Initial Data

lấy các cặp ảnh và mask

ảnh resize về 192x288

mask là binary
đầu ra imgs , masks

In [42]:
ff = os.listdir("thyroid")
folder = "thyroid/"
imgs = []
masks = []
labels = []
# h, w = 352 // 2, 512 // 2
for count, f in enumerate(ff):
    rd_num = np.random.uniform(0.7, 1)
    if f.find(".jpg") > 0:
        img = cv2.imread(folder + f)
        # img = cv2.resize(img, (w, h))
        ix_number = f.find("_")
        rep = f[ix_number + 1:ix_number + 2]
        if ix_number < 0:
            ix_number = f.find(".")
        number = f[:ix_number]
        if number == "127" or number == '54' or number == '165' or number == '176' or number == '203' or number == '205' or number == '166' or number == '197':
            continue
        # print(number)
        root = ET.parse(folder + number + ".xml").getroot()
        # print("so anh: ",len(root.findall("mark")))
        lb = (root.find("tirads").text)
        if lb in ('4a','4b','4c','5'):
            tt = 1
        else:
            tt = 0
        for neighbor in root.findall("mark"):
            if rep != neighbor.find("image").text:
                continue
            svg = neighbor.find("svg").text
            if svg is None:
                continue
            ss = eval(svg)
            for s in ss:
                contour = []
                for point in s["points"]:
                    x = point["x"]
                    y = point['y']
                    contour.append([[x, y]])
                contour = np.asarray(contour)
                rect = cv2.boundingRect(contour)
                x,y,w,h = rect
                blob= img[y-h//10:y+h+h//10,x-w//10:x+w+w//10]
                blob = cv2.resize(blob,(96,96))
                imgs.append(blob)
                labels.append(tt)
imgs = np.asarray(imgs,dtype = np.float32)
labels = np.asarray(labels,dtype = np.float32)
from sklearn.utils import shuffle
imgs, labels = shuffle(imgs, labels)

In [43]:
from keras.utils.np_utils import to_categorical  
labels = to_categorical(labels,2)
n = 550
x_train = imgs[:n]
x_dev = imgs[n:]

y_train = labels[:n]
y_dev = labels[n:]

In [ ]:
from sklearn.model_selection import train_test_split
 #from itertools import izip
generator_x = ImageDataGenerator(
    horizontal_flip = True,
    vertical_flip = True,
    #width_shift_range = 0.2,
    #height_shift_range = 0.2,
    #shear_range = 0.1,
    zoom_range = 0.1,
    rotation_range = 180,
    #fill_mode = 'reflect',
    #brightness_range = (0.9,1.1),
)
generator_y = ImageDataGenerator(
    horizontal_flip = True,
    vertical_flip = True,
    #width_shift_range = 0.2,
    #height_shift_range = 0.2,
    #shear_range = 0.1,
    zoom_range = 0.1,
    #brightness_range = (1.0,1.0),
    rotation_range = 180,
    #fill_mode = 'reflect'
)
seed = 1234
batch = 1
#X_train, X_test, y_train, y_test = train_test_split(imgs, masks, test_size=0.1)
x_gen = generator_x.flow(X_train, batch_size = batch, shuffle = False, seed = seed)
y_gen = generator_y.flow(y_train, batch_size = batch, shuffle = False, seed = seed)
train_generator = zip(x_gen, y_gen)

# Fit model

Epoch 60/80
500/500 [==============================] - 288s 576ms/step - loss: 0.3507 - acc: 0.9603 - dice_coef: 0.9069 - jaccard_coef: 0.8409 - val_loss: 0.4036 - val_acc: 0.9290 - val_dice_coef: 0.8515 - val_jaccard_coef: 0.7673

In [55]:
pretrain.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 102, 102, 3)  0           input_19[0][0]                   
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 48, 48, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 48, 48, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [65]:
input_ten=Input(shape=(96,96,3))
pretrain = app.ResNet50(include_top = False, weights='imagenet', input_tensor=input_ten)
pre_out=pretrain.get_layer("conv3_block4_out").output
flat=Flatten()(pre_out)
predict=Dense(256,activation='relu')(flat)
predict=Dense(2,activation='sigmoid')(predict)
model=Model(inputs=input_ten,outputs=predict)
sgd = Adam(lr = 1e-5)

model.compile(optimizer = sgd, loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

#lr_decay = lr_scheduler()
history = model.fit(x_train,y_train,batch_size=32, epochs = 50, 
                    validation_data = (x_dev, y_dev))

Epoch 1/50
18/18 [==============================] - 1s 50ms/step - loss: 0.8252 - accuracy: 0.5964 - val_loss: 0.7342 - val_accuracy: 0.5660
Epoch 2/50
18/18 [==============================] - 1s 28ms/step - loss: 0.5297 - accuracy: 0.7564 - val_loss: 0.7285 - val_accuracy: 0.5660
Epoch 3/50
18/18 [==============================] - 0s 28ms/step - loss: 0.3837 - accuracy: 0.8564 - val_loss: 0.9003 - val_accuracy: 0.5660
Epoch 4/50
18/18 [==============================] - 0s 28ms/step - loss: 0.2730 - accuracy: 0.9582 - val_loss: 0.7849 - val_accuracy: 0.5566
Epoch 5/50
18/18 [==============================] - 0s 28ms/step - loss: 0.2060 - accuracy: 0.9836 - val_loss: 1.1486 - val_accuracy: 0.5660
Epoch 6/50
18/18 [==============================] - 1s 28ms/step - loss: 0.1526 - accuracy: 0.9891 - val_loss: 0.8269 - val_accuracy: 0.5660
Epoch 7/50
18/18 [==============================] - 0s 28ms/step - loss: 0.1049 - accuracy: 1.0000 - val_loss: 0.7506 - val_accuracy: 0.5660
Epoch 8/50
18

Epoch 49/50
35/35 [==============================] - 1s 23ms/step - loss: 5.8039e-05 - accuracy: 1.0000 - val_loss: 2.5970 - val_accuracy: 0.7075

pre_out=pretrain.get_layer("conv3_block4_out").output
flat=Flatten()(pre_out)
predict=Dense(256,activation='relu')(flat)
predict=Dense(2,activation='sigmoid')(predict)

#visualize 

In [ ]:
val_main_output_dice_coefmodel = unet((352,352,3),2,"/content/drive/My Drive/weight_infection_acl/unet.23_0.72.h5")
mask_pred = model.predict(imgs)
mask_copy = mask_pred.copy()
mask_pred = heviside(mask_pred,0.01)
#mask_pred[mask_pred<0] = 0
#mask_pred[mask_pred>0] = 1

In [ ]:
plt.imshow(mask_pred[1,:,:,0],cmap= 'gray')
plt.show()    
plt.imshow(masks[1,:,:,0],cmap= 'gray')
plt.show()

In [ ]:
epison = 1/3.14159
Drc = epison / (3.14159 * (epison * epison + mask_copy[:,:,:,:] * mask_copy[:,:,:,:]))
print(K.mean(tf.keras.losses.binary_crossentropy(edge,Drc)))
plt.imshow(Drc[1,:,:,0],cmap= 'gray')
plt.xticks(np.arange(0, 353, 50))
plt.show()   
plt.imshow(edge[1,:,:,0],cmap= 'gray')
plt.show()
print(edge[1,:,:,0].max())

In [ ]:
x = masks[:,1:,:,:] - masks[:,:-1,:,:] # horizontal and vertical directions 
y = masks[:,:,1:,:] - masks[:,:,:-1,:]


delta_x = x[:,1:,:-2,:]**2
delta_y = y[:,:-2,1:,:]**2
delta_u = K.sqrt(K.abs(delta_x + delta_y))/np.sqrt(2)
print(delta_u.numpy().max())

plt.imshow(delta_u[1,:,:,0],cmap= 'gray')
plt.show()
plt.imshow(edge[1,:,:,0],cmap= 'gray')
plt.show()

In [ ]:
def dice_coef2(y_true, y_pred, smooth=1.0):
    '''Average dice coefficient per batch.'''
    axes = (1,2,3)
    intersection = K.sum(y_true * y_pred, axis=axes) #AhopM
    summation = K.sum(y_true + y_pred, axis=axes)
    
    return K.mean((2.0 * intersection + smooth) / (summation + smooth), axis=0)

def dice_coef_loss2(y_true, y_pred):
    return 1-dice_coef2(y_true, y_pred)

In [ ]:
model.load_weights("/content/drive/My Drive/weight_infection_acl/unet.55_0.74.h5")
mask_pred = model.predict(img_dev)
#mask_pred = heviside(mask_pred,epsilon=0.01)
mask_pred[mask_pred<0] = 0
mask_pred[mask_pred>0] = 1
print(dice_coef2(mask_dev,mask_pred))
plt.imshow(mask_pred[1,:,:,0],cmap= 'gray')
plt.show()    
plt.imshow(mask_dev[1,:,:,0],cmap= 'gray')
plt.show()

In [ ]:
import tensorflow.keras.applications as app
model = app.MobileNet()
for count,layer in enumerate(model.layers):
	# check for convolutional layer
	#if 'conv' not in layer.name:
	#	continue
	# get filter weights
	print(count, layer.name)
	#filters, biases = layer.get_weights()


In [ ]:
ixs = [2, 5, 9, 91, 94,97]
outputs = [model.layers[i].output for i in ixs]
model = Model(inputs=model.inputs, outputs=outputs)
pyplot.imshow(imgs[0,:,:,:].astype("uint8"),cmap='gray')
pyplot.show()
feature_maps = model.predict(imgs[:1,:,:,:])
# plot the output from each block
square = 2
for fmap in feature_maps:
    print(fmap.shape)
    ix = 1
    for _ in range(square):
        for _ in range(square):
            # specify subplot and turn of axis
            ax = pyplot.subplot(square, square, ix)
            ax.set_xticks([])
            ax.set_yticks([])
            # plot filter channel in grayscale
            if ix > fmap.shape[-1]:
                continue
            pyplot.imshow(fmap[0, :, :, ix-1], cmap='gray')
            ix += 1
	# show the figure
    pyplot.show()

In [ ]:
#plt.figure(1)
#plt.subplot(121,aspect='auto')
y=history.history['val_dice_coef']
plt.plot(history.history['dice_coef'])
plt.plot(history.history['val_dice_coef'])
plt.title('Model DCS')
plt.ylabel('DCS')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

#plt.subplot(122,aspect='auto')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

plt.show()
